In [2]:
import warnings
warnings.simplefilter('ignore')

import gc
import re
from collections import defaultdict, Counter

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from tqdm.auto import tqdm

In [3]:
df_sess = pd.read_csv('sessions_train.csv')
# df_prod = pd.read_csv('products_train.csv')

In [4]:
df_sess['locale'].unique()
# df_sess.columns

array(['DE', 'JP', 'UK', 'ES', 'FR', 'IT'], dtype=object)

In [5]:
# df_prod_task1 = df_prod[df_prod['locale'].isin(['UK', 'DE', 'JP'])]
# df_prod_task1

In [6]:
df_sess_task1 = df_sess[df_sess['locale'].isin(['UK', 'DE', 'JP'])]
df_sess_task1
# these can be decreased to smaller datasets to try faster

,prev_items,next_item,locale
0,['B09W9FND7K' 'B09JSPLN1M'],B09M7GY217,DE
1,['B076THCGSG' 'B007MO8IME' 'B08MF65MLV' 'B001B...,B001B4THSA,DE
2,['B0B1LGXWDS' 'B00AZYORS2' 'B0B1LGXWDS' 'B00AZ...,B0767DTG2Q,DE
3,['B09XMTWDVT' 'B0B4MZZ8MB' 'B0B7HZ2GWX' 'B09XM...,B0B4R9NN4B,DE
4,['B09Y5CSL3T' 'B09Y5DPTXN' 'B09FKD61R8'],B0BGVBKWGZ,DE
...,...,...,...
3272711,['B06XK89969' 'B01NGT5NF4' 'B00D5Z89C8' 'B07ZV...,B07VL2W1DR,UK
3272712,['B076M85W1K' 'B07L8792Q9' 'B095RW318L' 'B095R...,B095RQ2LCY,UK
3272713,['B00JQDIQRQ' 'B001O59QQE'],B088M5YT6Y,UK
3272714,['B07QMHMLJZ' 'B07FPYYMC4'],B07PFF814D,UK


In [7]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

import numpy as np
from scipy.sparse import csr_matrix

df = df_sess[df_sess['locale'] == 'IT'].head(100)
df

,prev_items,next_item,locale
3479324,['B09H2YTDV1' 'B07TTHHWTF' 'B0BCKC4RZD'],B0BCJHQC75,IT
3479325,['B09XGPHCG6' 'B09XGR9761' 'B09XGWZMRK' 'B09XG...,B09XGZJSVZ,IT
3479326,['B08RZCJM6Z' 'B08RZCJM6Z'],B0B81HH722,IT
3479327,['B01BJQ35WI' 'B01BJQ2060' 'B01618WO9O'],B00TKE3JKA,IT
3479328,['B094J8TNDB' 'B07Q5SYP2N'],B07Q833RCB,IT
...,...,...,...
3479419,['B07Y8GQMP5' 'B07Y8GCMLG' 'B09F3P3DQD' 'B09CK...,B09PRB9F9F,IT
3479420,['B0888NTP6Y' 'B0888NTP6Y' 'B07574GKDZ'],B0918G7MM9,IT
3479421,['B0B9N3RGMY' 'B089L1VYLQ'],B089KXDHXM,IT
3479422,['B083GP2T16' 'B083GPLPVW'],B08XXCM7PM,IT


In [17]:
# we will use just df_sess_task1 for our ar model

# Convert array of strings to list of items for each session
transactions = df['prev_items']

# Convert next_item to a list of length 1 for each session
next_items = df['next_item'].apply(lambda x: [x])

# print(transactions)
# Initialize and fit transaction encoder
te = TransactionEncoder()
te.fit(transactions)

# Transform transactions and next_items into binary matrix
X = te.transform(transactions)
# y = te.transform(next_items)
X = pd.DataFrame(X, columns=te.columns_)

# Generate frequent itemsets
min_support = 0.05 # Example support threshold
frequent_itemsets = apriori(X, min_support=min_support, use_colnames=False, max_len=2)
# print(frequent_itemsets)
# Generate association rules
min_confidence = 0.5 # Example confidence threshold
association_rules_df = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)
# print(association_rules_df)
# Make recommendations for a new session
new_session = {8, 3, 2,10} # Example viewed items
antecedents = frequent_itemsets['itemsets'].apply(lambda x: x.issubset(new_session))
# print(frequent_itemsets['itemsets'][120])
print(antecedents.shape, association_rules_df.shape)
# antecedents[5] = True
print(antecedents[antecedents].index)
# print(association_rules_df['antecedents'][1599])
# recommendations = association_rules_df['antecedents'].sort_values(by='confidence', ascending=False).iloc[0]['consequents']
# print(recommendations)


(770,) (880, 10)
Int64Index([2, 3, 8, 10, 115, 120, 122, 157, 159, 320], dtype='int64')
